# HK Covid-19 fortnightly visualization of cases using Uber H3 Spatial Index

In [ ]:
# Import all the important libraries for this project

In [1]:
# for pandas dataframes
import pandas as pd
# for numpy arrays
import numpy as np

# for json and geojson data libraries
import json
import geojson

from geojson import Feature, FeatureCollection

# to handle  data retrieval from https API
import urllib3
from urllib3 import request
import requests

# geopandas and geopy for geolocation
import geopandas
import geopy
from geopy.geocoders import Nominatim

# to handle file input and outputs
import io

# for data visualization
import matplotlib.pyplot as plt
%matplotlib inline
from mpltools import color
import plotly
import plotly.express as px

# geo-spatial index and visualization libraries
import h3
import folium
from folium import GeoJson
import branca.colormap as cm

In [3]:
# Set up dynamic date parameter to pass into HK Gov API string

In [2]:
from datetime import date, timedelta

today = date.today()-timedelta(1)
d = today.strftime("%Y%m%d")
print("d1 =", d)

d1 = 20210413


In [ ]:
# Pull data from HK Gov health website and pass into pandas dataframe

In [3]:
result=requests.get("https://api.data.gov.hk/v1/historical-archive/get-file?url=http%3A%2F%2Fwww.chp.gov.hk%2Ffiles%2Fmisc%2Fbuilding_list_eng.csv&time=" + d + "-0906")
result = result.content
data = pd.read_csv(io.StringIO(result.decode('utf-8')))

In [4]:
data.head()

,District,Building name,Last date of residence of the case(s),Related probable/confirmed cases
0,Eastern,"Block 7, Fullview Garden",NaN,11449
1,Yau Tsim Mong,The Kowloon Hotel,NaN,11450
2,Central & Western,Ramada Hong Kong Harbour View,NaN,11452
3,Yau Tsim Mong,Silka Seaview Hotel Hong Kong,NaN,11453
4,Yau Tsim Mong,Silka Seaview Hotel Hong Kong,NaN,11454


In [9]:
# mapping latitude and longitude to district names from HK Gov API data

In [5]:
location = [x for x in data['District'].unique().tolist() 
            if type(x) == str]
latitude = []
longitude =  []
for i in range(0, len(location)):
    # remove things that does not seem usefull here
    try:
        address = location[i] + ', Hong Kong, Hong Kong'
        geolocator = Nominatim(user_agent="ny_explorer")
        loc = geolocator.geocode(address)
        latitude.append(loc.latitude)
        longitude.append(loc.longitude)
    except:
        # in the case the geolocator does not work, then add nan element to list
        # to keep the right size
        latitude.append(np.nan)
        longitude.append(np.nan)
# create a dataframe with the locatio, latitude and longitude
df_ = pd.DataFrame({'District':location, 
                    'district_latitude': latitude,
                    'district_longitude':longitude})
# merge on Restaurant_Location with rest_df to get the column 
data1 = data.merge(df_, on='District', how='outer')

In [11]:
# mapping latitude and longitude to building names from HK Gov API data

In [6]:
location = [x for x in data['Building name'].unique().tolist() 
            if type(x) == str]
latitude = []
longitude =  []
for i in range(0, len(location)):
    # remove things that does not seem usefull here
    try:
        address = location[i] + ', Hong Kong, Hong Kong'
        geolocator = Nominatim(user_agent="ny_explorer")
        loc = geolocator.geocode(address)
        latitude.append(loc.latitude)
        longitude.append(loc.longitude)
    except:
        # in the case the geolocator does not work, then add nan element to list
        # to keep the right size
        latitude.append(np.nan)
        longitude.append(np.nan)
# create a dataframe with the locatio, latitude and longitude
df_ = pd.DataFrame({'Building name':location, 
                    'building_latitude': latitude,
                    'building_longitude':longitude})
# merge on Restaurant_Location with rest_df to get the column 
data2 = data1.merge(df_, on='Building name', how='left')

In [7]:
data2.head()

,District,Building name,Last date of residence of the case(s),Related probable/confirmed cases,district_latitude,district_longitude,building_latitude,building_longitude
0,Eastern,"Block 7, Fullview Garden",NaN,11449,22.273078,114.233594,22.263548,114.251822
1,Eastern,Continental Mansion,NaN,11477,22.273078,114.233594,22.289805,114.195564
2,Eastern,Ramada Hong Kong Grand View,NaN,11480,22.273078,114.233594,NaN,NaN
3,Eastern,Continental Mansion,NaN,11491,22.273078,114.233594,22.289805,114.195564
4,Eastern,Ramada Hong Kong Grand View,NaN,11506,22.273078,114.233594,NaN,NaN


In [ ]:
#populate missing values at micro (building) level with macro (district) coordinates

In [8]:
#missing building latitude
data2['building_latitude'] = data2.apply(
    lambda row: row['district_latitude'] if np.isnan(row['building_latitude']) else row['building_latitude'],
    axis=1)
#missing building longitude
data2['building_longitude'] = data2.apply(
    lambda row: row['district_longitude'] if np.isnan(row['building_longitude']) else row['building_longitude'],
    axis=1)

In [9]:
data2.head()

,District,Building name,Last date of residence of the case(s),Related probable/confirmed cases,district_latitude,district_longitude,building_latitude,building_longitude
0,Eastern,"Block 7, Fullview Garden",NaN,11449,22.273078,114.233594,22.263548,114.251822
1,Eastern,Continental Mansion,NaN,11477,22.273078,114.233594,22.289805,114.195564
2,Eastern,Ramada Hong Kong Grand View,NaN,11480,22.273078,114.233594,22.273078,114.233594
3,Eastern,Continental Mansion,NaN,11491,22.273078,114.233594,22.289805,114.195564
4,Eastern,Ramada Hong Kong Grand View,NaN,11506,22.273078,114.233594,22.273078,114.233594


In [ ]:
# code to take HK Gov Covid-19 and convert to hexagon tiles aggregated by latitude 
# and longitude and plot on a HK map

In [10]:
# Code to aggregate district latitude and longitude values and assign geometry shape 
# of polygon based on coordinates

def counts_by_hexagon_district(df, resolution):
    """
    Use h3.geo_to_h3 to index each data point into the spatial index of the specified resolution.
    Use h3.h3_to_geo_boundary to obtain the geometries of these hexagons
    
    Ex counts_by_hexagon(data, 9)
    """
    df = df[["district_latitude", "district_longitude"]]
    
    df["hex_id"] = df.apply(lambda row: h3.geo_to_h3(row["district_latitude"], row["district_longitude"], resolution), axis = 1)
    
    df_aggreg = df.groupby(by = "hex_id").size().reset_index()
    df_aggreg.columns = ["hex_id", "value"]
    
    df_aggreg["geometry"] =  df_aggreg.hex_id.apply(lambda x: 
                                                           {    "type" : "Polygon",
                                                                 "coordinates": 
                                                                [h3.h3_to_geo_boundary(h=x,geo_json=True)]
                                                            }
                                                        )
    
    return df_aggreg

# Code to aggregate building name latitude and longitude values and assign geometry shape 
# of polygon based on coordinates

def counts_by_hexagon_building(df, resolution):
    """
    Use h3.geo_to_h3 to index each data point into the spatial index of the specified resolution.
    Use h3.h3_to_geo_boundary to obtain the geometries of these hexagons
    
    Ex counts_by_hexagon(data, 9)
    """
    df = df[["building_latitude", "building_longitude"]]
    
    df["hex_id"] = df.apply(lambda row: h3.geo_to_h3(row["building_latitude"], row["building_longitude"], resolution), axis = 1)
    
    df_aggreg = df.groupby(by = "hex_id").size().reset_index()
    df_aggreg.columns = ["hex_id", "value"]
    
    df_aggreg["geometry"] =  df_aggreg.hex_id.apply(lambda x: 
                                                           {    "type" : "Polygon",
                                                                 "coordinates": 
                                                                [h3.h3_to_geo_boundary(h=x,geo_json=True)]
                                                            }
                                                        )
    
    return df_aggreg


# Code to create json file with Feature inputs based on values and geometry to be
# input in GeoJson for plotting on HK map

def hexagons_dataframe_to_geojson(df_hex, file_output = None):
    """
    Produce the GeoJSON for a dataframe that has a geometry column in geojson 
    format already, along with the columns hex_id and value
    
    Ex counts_by_hexagon(data)
    """    
    list_features = []
    
    for i,row in df_hex.iterrows():
        feature = Feature(geometry = row["geometry"] , id=row["hex_id"], properties = {"value" : row["value"]})
        list_features.append(feature)
        
    feat_collection = FeatureCollection(list_features)
    
    geojson_result = json.dumps(feat_collection)
    
    #optionally write to file
    if file_output is not None:
        with open(file_output,"w") as f:
            json.dump(feat_collection,f)
    
    return geojson_result

# Function to plot cases as polygons on HK map

def choropleth_map(df_aggreg, border_color = 'black', fill_opacity = 0.7, initial_map = None, with_legend = False,
                   kind = "linear"):
    
    """
    Creates choropleth maps given the aggregated data.
    """    
    #colormap
    min_value = df_aggreg["value"].min()
    max_value = df_aggreg["value"].max()
    m = round((min_value + max_value ) / 2 , 0)
    
    #take resolution from the first row
    res = h3.h3_get_resolution(df_aggreg.loc[0,'hex_id'])
    
    if initial_map is None:
        initial_map = folium.Map(location= [22.280887, 114.157500], zoom_start=12, tiles="cartodbpositron", 
                attr= '© <a href="http://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors © <a href="http://cartodb.com/attributions#basemaps">CartoDB</a>' 
            )
        

    #the colormap 
    #color names accepted https://github.com/python-visualization/branca/blob/master/branca/_cnames.json
    if kind == "linear":
        custom_cm = cm.LinearColormap(['pink','mediumvioletred','red'], vmin=min_value, vmax=max_value)
    elif kind == "filled_nulls":
        custom_cm = cm.LinearColormap(['white'], 
                                      index=[0,min_value,m,max_value],vmin=min_value,vmax=max_value)
   

    #create geojson data from dataframe
    geojson_data = hexagons_dataframe_to_geojson(df_hex = df_aggreg)
    
    #plot on map
    name_layer = "Choropleth " + str(res)
    if kind != "linear":
        name_layer = name_layer + kind
        
    GeoJson(
        geojson_data,
        style_function=lambda feature: {
            'fillColor': custom_cm(feature['properties']['value']),
            'color': border_color,
            'weight': 1,
            'fillOpacity': fill_opacity 
        }, 
        name = name_layer
    ).add_to(initial_map)
    #add legend (not recommended if multiple layers)
    if with_legend == True:
        custom_cm.add_to(initial_map)
        
    
    return initial_map

In [ ]:
# pass our HK Gov Covid-19 dataset into aggregation function
# View at District Level

In [11]:
df_aggreg = counts_by_hexagon_district(df = data2, resolution = 7)
df_aggreg.sort_values(by = "value", ascending = False, inplace = True)

<ipython-input-10-a7a53ee5aef4>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["hex_id"] = df.apply(lambda row: h3.geo_to_h3(row["district_latitude"], row["district_longitude"], resolution), axis = 1)


In [12]:
choropleth_map(df_aggreg, with_legend = True)

In [ ]:
# pass our HK Gov Covid-19 dataset into aggregation function
# View at Buidling Name Level

In [13]:
df_aggreg = counts_by_hexagon_building(df = data2, resolution = 8)
df_aggreg.sort_values(by = "value", ascending = False, inplace = True)

<ipython-input-10-a7a53ee5aef4>:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["hex_id"] = df.apply(lambda row: h3.geo_to_h3(row["building_latitude"], row["building_longitude"], resolution), axis = 1)


In [14]:
choropleth_map(df_aggreg, with_legend = True)

# Thank You!